In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pylab
import statsmodels.api as sm
from pprint import pprint as pp
%matplotlib inline

In [6]:
from scipy import stats
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, Normalizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [7]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [8]:
def avg_training_score_grading(x):
    grade = 0;
    if x < 59 :
        grade = 0
    elif x < 79:
        grade = 1
    else :
        grade = 2
    return grade    

In [9]:
def no_of_trainings_grade(x):
    grade = 0
    if x <= 5:
        grade = 0
    elif x <= 7:
        grade = 1
    else :
        grade = 2
    return grade   

In [10]:
def education_encode(x):
    x = str(x)
    x = ''.join(list(x)[0:4])
    grade = np.NaN
    if x[0:4] == 'Bach':
        grade = 1
    elif x[0:4] == 'Mast':
        grade = 2
    elif x[0:4] == 'Belo':
        grade = 0
    return grade 

In [11]:
def region_encode(x):
    return int(x[len('region_'):])

In [12]:
def age_bining(x):
    x = int(x)
    x = (x//5)*5
    return x    

In [13]:
df_train['age'] = df_train['age'].apply(age_bining)
df_test['age'] = df_test['age'].apply(age_bining)

In [14]:
df_train['region'] = df_train['region'].apply(region_encode)
df_test['region'] = df_test['region'].apply(region_encode)

In [15]:
df_train['no_of_trainings'] = df_train['no_of_trainings'].apply(no_of_trainings_grade)
df_test['no_of_trainings'] = df_test['no_of_trainings'].apply(no_of_trainings_grade)

In [16]:
df_train['avg_training_score'] = df_train['avg_training_score'].apply(avg_training_score_grading)
df_test['avg_training_score'] = df_test['avg_training_score'].apply(avg_training_score_grading)

In [17]:
df_train['education'] = df_train['education'].apply(education_encode)
df_test['education'] = df_test['education'].apply(education_encode)

In [18]:
imputer = Imputer(missing_values='NaN', strategy="most_frequent", axis=0)
train_previous_year_rating = imputer.fit_transform(df_train.previous_year_rating.values.reshape(-1, 1))
df_train['previous_year_rating'] = train_previous_year_rating
test_previous_year_rating = imputer.transform(df_test.previous_year_rating.values.reshape(-1, 1))
df_test['previous_year_rating'] = test_previous_year_rating

In [19]:
df_train['education'] = df_train['education'].replace(r'', np.NaN)
df_test['education'] = df_test['education'].replace(r'', np.NaN)

In [20]:
df_train['education'] = df_train['education'].astype(object)
df_test['education'] = df_test['education'].astype(object)

In [21]:
imputer = Imputer(missing_values='NaN', strategy="most_frequent", axis=0)
train_education_filled = imputer.fit_transform(df_train.education.values.reshape(-1, 1))
df_train['education'] = train_education_filled
test_education_filled = imputer.transform(df_test.education.values.reshape(-1, 1))
df_test['education'] = test_education_filled

In [22]:
dict_labelencode={}
for col in ['department', 'region', 'gender', 'recruitment_channel', 'age', 'no_of_trainings', 
                    'length_of_service', 'KPIs_met >80%', 'awards_won?', 'avg_training_score', 
                    'previous_year_rating', 'education']:
#for col in ['department', 'gender', 'recruitment_channel']:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    df_test[col] = le.transform(df_test[col])
    dict_labelencode[col] = le.classes_
    print("One hot encoding  for -----> ", col)                
    onehotencoder = OneHotEncoder(categorical_features = 'all')
    dummy_var = onehotencoder.fit_transform(df_train[col].values.reshape(-1,1)).toarray()
    valueList = dict_labelencode[col]
    i = 0 
    for value in valueList:
        print("One hot encoding  for value-----> ", value) 
        newcolumn = col + str("_") + str(value)
        print("One hot encoding  for label class -----> ", newcolumn)
        df_train[newcolumn] = dummy_var[:, i]
        i = i+1
    df_train = df_train.drop([newcolumn], axis=1)    
    df_train = df_train.drop([col], axis=1)
    print("dropped column -----> ", newcolumn, col)
    print("############## test data ###################")
    dummy_var = onehotencoder.transform(df_test[col].values.reshape(-1,1)).toarray()
    valueList = dict_labelencode[col]
    i = 0 
    for value in valueList:
        print("test One hot encoding  for value-----> ", value) 
        newcolumn = col + str("_") + str(value)
        print("test One hot encoding  for label class -----> ", newcolumn)
        df_test[newcolumn] = dummy_var[:, i]
        i = i+1
    df_test = df_test.drop([newcolumn], axis=1)    
    df_test = df_test.drop([col], axis=1)
    print("test dropped column -----> ", newcolumn, col)
    
    #df_test = df_test.drop([col], axis=1)
    

One hot encoding  for ----->  department
One hot encoding  for value----->  Analytics
One hot encoding  for label class ----->  department_Analytics
One hot encoding  for value----->  Finance
One hot encoding  for label class ----->  department_Finance
One hot encoding  for value----->  HR
One hot encoding  for label class ----->  department_HR
One hot encoding  for value----->  Legal
One hot encoding  for label class ----->  department_Legal
One hot encoding  for value----->  Operations
One hot encoding  for label class ----->  department_Operations
One hot encoding  for value----->  Procurement
One hot encoding  for label class ----->  department_Procurement
One hot encoding  for value----->  R&D
One hot encoding  for label class ----->  department_R&D
One hot encoding  for value----->  Sales & Marketing
One hot encoding  for label class ----->  department_Sales & Marketing
One hot encoding  for value----->  Technology
One hot encoding  for label class ----->  department_Technology
d

In [68]:
df_train.columns

Index(['employee_id', 'is_promoted', 'department_Analytics',
       'department_Finance', 'department_HR', 'department_Legal',
       'department_Operations', 'department_Procurement', 'department_R&D',
       'department_Sales & Marketing', 'region_1', 'region_2', 'region_3',
       'region_4', 'region_5', 'region_6', 'region_7', 'region_8', 'region_9',
       'region_10', 'region_11', 'region_12', 'region_13', 'region_14',
       'region_15', 'region_16', 'region_17', 'region_18', 'region_19',
       'region_20', 'region_21', 'region_22', 'region_23', 'region_24',
       'region_25', 'region_26', 'region_27', 'region_28', 'region_29',
       'region_30', 'region_31', 'region_32', 'region_33', 'gender_f',
       'recruitment_channel_other', 'recruitment_channel_referred', 'age_20',
       'age_25', 'age_30', 'age_35', 'age_40', 'age_45', 'age_50', 'age_55',
       'no_of_trainings_0', 'no_of_trainings_1', 'length_of_service_1',
       'length_of_service_2', 'length_of_service_3', 'len

In [69]:
df_test.columns

Index(['employee_id', 'department_Analytics', 'department_Finance',
       'department_HR', 'department_Legal', 'department_Operations',
       'department_Procurement', 'department_R&D',
       'department_Sales & Marketing', 'region_1', 'region_2', 'region_3',
       'region_4', 'region_5', 'region_6', 'region_7', 'region_8', 'region_9',
       'region_10', 'region_11', 'region_12', 'region_13', 'region_14',
       'region_15', 'region_16', 'region_17', 'region_18', 'region_19',
       'region_20', 'region_21', 'region_22', 'region_23', 'region_24',
       'region_25', 'region_26', 'region_27', 'region_28', 'region_29',
       'region_30', 'region_31', 'region_32', 'region_33', 'gender_f',
       'recruitment_channel_other', 'recruitment_channel_referred', 'age_20',
       'age_25', 'age_30', 'age_35', 'age_40', 'age_45', 'age_50', 'age_55',
       'no_of_trainings_0', 'no_of_trainings_1', 'length_of_service_1',
       'length_of_service_2', 'length_of_service_3', 'length_of_service_

In [23]:
X1 = df_train.loc[:,['department_Analytics',
       'department_Finance', 'department_HR', 'department_Legal',
       'department_Operations', 'department_Procurement', 'department_R&D',
       'department_Sales & Marketing', 'region_1', 'region_2', 'region_3',
       'region_4', 'region_5', 'region_6', 'region_7', 'region_8', 'region_9',
       'region_10', 'region_11', 'region_12', 'region_13', 'region_14',
       'region_15', 'region_16', 'region_17', 'region_18', 'region_19',
       'region_20', 'region_21', 'region_22', 'region_23', 'region_24',
       'region_25', 'region_26', 'region_27', 'region_28', 'region_29',
       'region_30', 'region_31', 'region_32', 'region_33', 'gender_f',
       'recruitment_channel_other', 'recruitment_channel_referred', 'age_20',
       'age_25', 'age_30', 'age_35', 'age_40', 'age_45', 'age_50', 'age_55',
       'no_of_trainings_0', 'no_of_trainings_1', 'length_of_service_1',
       'length_of_service_2', 'length_of_service_3', 'length_of_service_4',
       'length_of_service_5', 'length_of_service_6', 'length_of_service_7',
       'length_of_service_8', 'length_of_service_9', 'length_of_service_10',
       'length_of_service_11', 'length_of_service_12', 'length_of_service_13',
       'length_of_service_14', 'length_of_service_15', 'length_of_service_16',
       'length_of_service_17', 'length_of_service_18', 'length_of_service_19',
       'length_of_service_20', 'length_of_service_21', 'length_of_service_22',
       'length_of_service_23', 'length_of_service_24', 'length_of_service_25',
       'length_of_service_26', 'length_of_service_27', 'length_of_service_28',
       'length_of_service_29', 'length_of_service_30', 'length_of_service_31',
       'length_of_service_32', 'length_of_service_33', 'length_of_service_34',
       'KPIs_met >80%_0', 'awards_won?_0', 'avg_training_score_0',
       'avg_training_score_1', 'previous_year_rating_1.0',
       'previous_year_rating_2.0', 'previous_year_rating_3.0',
       'previous_year_rating_4.0', 'education_0.0', 'education_1.0']].values
y1 = df_train.loc[:,['is_promoted']].values
#y_train = df_train.loc[:,['is_promoted']].values

X2 = df_test.loc[:,['department_Analytics', 'department_Finance',
       'department_HR', 'department_Legal', 'department_Operations',
       'department_Procurement', 'department_R&D',
       'department_Sales & Marketing', 'region_1', 'region_2', 'region_3',
       'region_4', 'region_5', 'region_6', 'region_7', 'region_8', 'region_9',
       'region_10', 'region_11', 'region_12', 'region_13', 'region_14',
       'region_15', 'region_16', 'region_17', 'region_18', 'region_19',
       'region_20', 'region_21', 'region_22', 'region_23', 'region_24',
       'region_25', 'region_26', 'region_27', 'region_28', 'region_29',
       'region_30', 'region_31', 'region_32', 'region_33', 'gender_f',
       'recruitment_channel_other', 'recruitment_channel_referred', 'age_20',
       'age_25', 'age_30', 'age_35', 'age_40', 'age_45', 'age_50', 'age_55',
       'no_of_trainings_0', 'no_of_trainings_1', 'length_of_service_1',
       'length_of_service_2', 'length_of_service_3', 'length_of_service_4',
       'length_of_service_5', 'length_of_service_6', 'length_of_service_7',
       'length_of_service_8', 'length_of_service_9', 'length_of_service_10',
       'length_of_service_11', 'length_of_service_12', 'length_of_service_13',
       'length_of_service_14', 'length_of_service_15', 'length_of_service_16',
       'length_of_service_17', 'length_of_service_18', 'length_of_service_19',
       'length_of_service_20', 'length_of_service_21', 'length_of_service_22',
       'length_of_service_23', 'length_of_service_24', 'length_of_service_25',
       'length_of_service_26', 'length_of_service_27', 'length_of_service_28',
       'length_of_service_29', 'length_of_service_30', 'length_of_service_31',
       'length_of_service_32', 'length_of_service_33', 'length_of_service_34',
       'KPIs_met >80%_0', 'awards_won?_0', 'avg_training_score_0',
       'avg_training_score_1', 'previous_year_rating_1.0',
       'previous_year_rating_2.0', 'previous_year_rating_3.0',
       'previous_year_rating_4.0', 'education_0.0', 'education_1.0']].values

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.20, random_state = 0)

In [25]:
df_train.describe()

,employee_id,is_promoted,department_Analytics,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,...,KPIs_met >80%_0,awards_won?_0,avg_training_score_0,avg_training_score_1,previous_year_rating_1.0,previous_year_rating_2.0,previous_year_rating_3.0,previous_year_rating_4.0,education_0.0,education_1.0
count,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000,...,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000,54808.000000
mean,39195.830627,0.085170,0.097650,0.046271,0.044118,0.018957,0.207050,0.130236,0.018227,0.307254,...,0.648026,0.976828,0.434316,0.352503,0.113542,0.077087,0.414939,0.180211,0.014688,0.712998
std,22586.581449,0.279137,0.296844,0.210072,0.205358,0.136375,0.405195,0.336566,0.133774,0.461360,...,0.477590,0.150450,0.495671,0.477754,0.317257,0.266732,0.492716,0.384367,0.120300,0.452367
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19669.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,39225.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,58730.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,78298.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Bagging Classifier

In [85]:
clf_tree=DecisionTreeClassifier(criterion='entropy', max_depth=20)
bag_classifier = BaggingClassifier(base_estimator=clf_tree, n_estimators=20)
bag_classifier.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=20,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=20, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [86]:
bg_y_pred = bag_classifier.predict(X_test)
print(accuracy_score(y_test, bg_y_pred))
print(f1_score(y_test, bg_y_pred))

0.932767743113
0.403238866397


In [87]:
bg_y_pred2 = bag_classifier.predict(X2)
df_test['is_promoted'] = bg_y_pred2
header = ['employee_id', 'is_promoted']
df_test[header].to_csv('bag_submission.csv', columns = header, index=False)

In [26]:
for max_depth in [20,30,40,50,60,70,80,90]:
    for min_samples_split in range(10,250,20) :
        clf_tree=DecisionTreeClassifier(criterion='entropy', max_depth=max_depth, min_samples_split=min_samples_split)
        bag_classifier = BaggingClassifier(base_estimator=clf_tree, n_estimators=20)
        bag_classifier.fit(X_train, y_train)
        bg_y_pred = bag_classifier.predict(X_test)
        print("########Ac",max_depth, min_samples_split,accuracy_score(y_test, bg_y_pred))
        print("########F1",max_depth, min_samples_split, f1_score(y_test, bg_y_pred))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


########Ac 20 10 0.9352307973
########F1 20 10 0.415156507414


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


########Ac 20 30 0.934865900383
########F1 20 30 0.405


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

########Ac 90 190 0.933862433862
########F1 90 190 0.398340248963


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


########Ac 90 210 0.933953658092
########F1 90 210 0.399668325041


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


########Ac 90 230 0.934044882321
########F1 90 230 0.4
